# Narnian DEMO
During this experience, we will face some classical tasks in signal processing to introduce the world of NARNIAN. This tutorial will help the user to understand some basic interactions that can happen among different Agents, presenting a framework that is general and open to customization. We will define the Environment, its Streams and the behaviors of its inhabitant by defining Finite State Machines and we will also visualize the interactions among those agents through an online viewer.

In the [first example](#One-signal,-one-repetition) we will show the case in which the Teacher shows the Student a single stream and, after this single repetition, asks him to repeat it. During the [second example](#Fixed-Playlist), the Teacher alternates two signals and finally ask the Student to repeat both of them. Finally, in the [last example](#Complete-Learning-Cycle) we will evaluate the simplest complete scenario in NARNIAN, where the Teacher asks the Student to both replicate the signals and to describe their own output. The lesson from the Teacher is repeated until the Student doesn't met a threshold on a given metric, set by the Teacher. Once the Student reaches the required level on the first signal, the Teacher moves to the second one.

### Prepare the Colab notebook

1\. Check if we are in Colab and set the default port for the Server.

In [ ]:
import sys

IN_COLAB = "google.colab" in sys.modules
PORT = 5001

The next steps are only required if you want to run this notebook in Google Colab. **IF YOU ARE RUNNING LOCALLY THE NOTEBOOK YOU DON'T NEED TO EXECUTE THE FOLLOWING CELLS**.\
2\. Clone the repo (don't clone in narnian/ because Colab doesn't like it)

In [ ]:
if IN_COLAB:
    !git clone https://{github_token}@github.com/mela64/narnian.git demo
    %cd demo
    !pip install -q -r requirements.txt

3\. To serve the Flask server from Colab we need to use a third party service named ngrok:\
To use it follow these steps:
- [sign up](https://dashboard.ngrok.com/) to ngrok and get your `Authtoken` (you can get one on the left column of the Dashboard under "Identity & Access");
- copy that token in the Colab Secrets (the "key" icon on the left), name it "NGROK_TOKEN" and give it the access to the notebook.

In [ ]:
if IN_COLAB:
    !pip install -q pyngrok
    
    from pyngrok import ngrok
    from google.colab import userdata
    
    ngrok_token = userdata.get('NGROK_TOKEN')
    ngrok.set_auth_token(ngrok_token)
    # start the ngrok web app listening to the specified PORT
    public_url = ngrok.connect(str(PORT)).public_url
    print(public_url)

The last cell should have printed the link to the ngrok Web App, we can click it but it won't show nothing if we don't start the Narnian Server before. Now we can go on with the execution of the remaining cells. **IF YOU ARE RUNNING LOCALLY THE NOTEBOOK EXECUTE CELLS FROM HERE**.

### Load Dependencies

In [ ]:
import torch
from narnian.model import Model
from narnian.attributes import Attributes
from modules.networks import PredRNN
from modules.networks import GenCTBEInitStateBZeroInput, set_seed
from narnian.server import Server
from narnian.streams import Stream
from narnian.model import EmptyModel
from basic.basic_agent import BasicAgent
from basic.basic_streams import Sin, Square, CombSin
from basic.basic_environment import BasicEnvironment
from basic.basic_model_gen4all_pred4all_gd import BasicModel

## One signal, one repetition

In this first simple case, the Teacher shows 1000 samples of a target signal and immediately asks the Student to reproduce it.

### Define the Environment

We create the environment with name 'env' and we also set the title for visualization as 'Demo Signal Sandbox'.

In [ ]:
env = BasicEnvironment("env", title="Demo Signal Sandbox")
env.print_enabled = False  # turn off logging just to speed up the notebook execution
device = torch.device("cpu")

Then we define a new stream which is a combination of 3 Sine waves with random frequencies.\
A stream is characterized by a name, its source (the enviroment in this case) and the stream itself.

In [ ]:
env.add_stream(Stream.create(name="3sin", creator=env.name, stream=CombSin(f_cap=0.06, c_cap=0.8, delta=0.1, order=3)))

The Environment has 3 fundamental roles:
  1. to create its own streams and enable them to be experienced by agents;
  2. to ensure all the agents are aware of the existing streams;
  3. to ensure all the agents know each other.

In [ ]:
env.add_transit("init", "streams_enabled", action="enable_all_streams")
env.add_transit("streams_enabled", "streams_sent", action="send_streams_to_all")
env.add_transit("streams_sent", "ready", action="send_agents_to_all")

### Define the Teacher agent

We define the agent that will act the role of the Teacher in this demo.

Since each agent is characterized by a name, a model and its autority within the environment, we can now define the teacher agent as follows:
- Name: 'Teacher'
- Model: A dummy model
- Authority: 1 (the highest)

In [ ]:
teacher = BasicAgent("Teacher", model=EmptyModel(), authority=1.0)
teacher.print_enabled = False  # turn off logging just to speed up the notebook execution

Let's define the FSM of the teacher agent...

1\. Let the teacher agent know that the streams are now available. (init -> got_streams) _Rember that in the enviroment FSM we have enabled the streams_

In [ ]:
teacher.add_transit("init", "got_streams", action="get_streams")

2\. Now the Teacher is aware of the streams but doesn't know the other agents. 

In [ ]:
teacher.add_transit("got_streams", "got_agents", action="get_agents")

3\. The teacher, after knowing the other agents, is ready to record the streams flowing in the enviroment.

To identify the streams, we can use the `stream_hash` attribute, in this specific case the associated hash is "env:3sin"
We decide to record 1000 samples of the stream.

In [ ]:
teacher.add_transit("got_agents", "recording1", action="record", args={"stream_hash": env.name + ":3sin", "steps": 1000})

The teacher, after recording the streams, start to find for agents to engage.
_(Since this action does not imply any change in the FSM, we add a state action with the state recording1)_

`min_auth` and the `max_auth` are set to 0.0, meaning that we are waiting for agent with autority strictly equal to 0

In [ ]:
teacher.add_state_action("recording1", action="find_agent_to_engage", args={"min_auth": 0.0, "max_auth": 0.0})

4\. When students are found the teacher starts an engagement with them.

In [ ]:
teacher.add_transit("recording1", "student_found", action="send_engagement")

5\. All the students which answer with a positive feedback are then engaged in the learning process.

In [ ]:
teacher.add_transit("student_found", "student_engaged", action="got_engagement")

6\. The streams are shared with the students.

In [ ]:
teacher.add_transit("student_engaged", "stream_shared", action="share_streams")

7\. The teacher asks the students to learn the function (in this case the sin).

We set the input-output/descriptor stream to the stream hash of the teacher.
We ask to learn for 1000 steps

In [ ]:
teacher.add_transit("stream_shared", "asked_learn", action="ask_learn_gen",
                    args={"du_hash": teacher.name + ":recorded1", "yhat_hash": teacher.name + ":recorded1",
                          "dhat_hash": teacher.name + ":recorded1", "ask_steps": 1000})

8\. We end up the learning stage and we ask the students to generate the function.

In [ ]:
teacher.add_transit("asked_learn", "done_learn", action="done_learn_gen")
teacher.add_transit("done_learn", "asked_gen", action="ask_gen",
                    args={"du_hash": teacher.name + ":recorded1", "dhat_hash": teacher.name + ":recorded1", "ask_steps": 1000})

9\. We end up the process and move to the end state.

In [ ]:
teacher.add_transit("asked_gen", "finished", action="done_gen")

At this point we can add the Teacher to the environment.

In [ ]:
env.add_agent(teacher)

### Define a student agent

The Student is nothing but a different instance of a NARNIAN Agent, so it will be defined similarly to the Teacher case. The main difference is that in this case we will use the another Model that we'll inspect later and the Student's authority will be set to 0:
- Name: 'Student'
- Model: BasicModel
- Authority: 0 (the lowest)

In [ ]:
student = BasicAgent("Student", model=BasicModel(attributes=env.shared_attributes, lr=0.001, device=device, seed=4), authority=0.0)
student.print_enabled = False  # turn off logging just to speed up the notebook execution

We now set its FSM...

1. the Student starts by acquiring the information of which streams are circulating in the enviroment;
2. the Student knows who are the other agents in the environment;
3. the Student is ready to engage with a Teacher (notice now that we are looking for `min_auth` and `max_auth` equal to 1);
4. after the engagement with a Teacher, the Student receives information about the streams available from that Teacher;
5. the Student go through the learning phase;
6. the Student generates back the signal. 

In [ ]:
student.add_transit("init", "got_streams", action="get_streams")
student.add_transit("got_streams", "got_agents", action="get_agents")
student.add_transit("got_agents", "teacher_engaged", action="get_engagement", args={"min_auth": 1.0, "max_auth": 1.0})
student.add_transit("teacher_engaged", "got_teacher_streams", action="get_streams")
student.add_transit("got_teacher_streams", "learning", action="do_learn_gen")
student.add_transit("got_teacher_streams", "generated", action="do_gen")
student.add_transit("learning", "got_teacher_streams", action="nop")
student.add_transit("generated", "got_teacher_streams", action="nop")

#### Inside the Student Agent

Behind every agent there is a Model that generally consists of a **generator** and a **predictor**. The Teacher used a placeholder `EmptyModel` that does nothing; for the Student we use the `BasicModel` composed by the following parts:
- `CTBE`: A Structured State Space Model exploiting Unitary matrices to produce perpetual oscillations (used as *Generator*);
- `PredRNN`: A simple rnn model processing the *Generator*'s output online to produce its descriptor (used as *Predictor*).

Along the model's definition, BasicModel includes also the `learn` method which, in this specific case, it is implemented as a common online GD training step.

Add the Student to the environment and start a server to visualize it. Then run the simulation.

In [ ]:
env.add_agent(student)
Server(env=env, port=PORT)
env.run()

## Fixed Playlist

<a id='second_example'></a>

In this case, we are creating a Sandbox in which the Teacher has 2 different stream samples available for its Students.\
The lesson is structured as a playlist in which the Teacher provides the supervision, alternating the two signals.\
After 5 repetitions of each signal, the Student is asked to reproduce both of them.

### Adapt the Environment

We need to:
1. remove the Agents before we create the new ones;
2. add the new signal;
3. reset the state of the Environment to the Init state of its FSM.

In [ ]:
env.remove_all_agents()
env.add_stream(Stream.create(name="square", creator=env.name, stream=Square(freq=0.06, ampl=0.5, phase=0.5, delta=0.1)))
env.behav.reset_state()

### Create the new Teacher

We start as in the previous case but then we also record the second signal.

In [ ]:
teacher = BasicAgent("Teacher", model=EmptyModel(), authority=1.0)
teacher.print_enabled = False  # turn off logging just to speed up the notebook execution
teacher.add_transit("init", "got_streams", action="get_streams")
teacher.add_transit("got_streams", "got_agents", action="get_agents")
teacher.add_transit("got_agents", "recording1", action="record", args={"stream_hash": env.name + ":3sin", "steps": 1000})
teacher.add_transit("recording1", "recording2", action="record", args={"stream_hash": env.name + ":square", "steps": 1000})

#### Create the Playlist

Once all the streams have been recorded we can arrange the playlist, setting 5 repetitions supervised from the Teacher and a final request for generation (5+1).

In [ ]:
teacher.add_transit("recording2", "playlist_ready", action="set_pref_streams",
                    args={"stream_hashes": [teacher.name + ":recorded1", teacher.name + ":recorded2"], "repeat": 5+1})

teacher.add_state_action("playlist_ready", action="find_agent_to_engage", args={"min_auth": 0.0, "max_auth": 0.0})

Then we engage the student before we can start the lesson.

In [ ]:
teacher.add_transit("playlist_ready", "student_found", action="send_engagement")
teacher.add_transit("student_found", "playlist_ready", action="nop")
teacher.add_transit("student_found", "student_engaged", action="got_engagement")
teacher.add_transit("student_engaged", "stream_shared", action="share_streams")

We ask the Student to learn to generate.

In [ ]:
teacher.add_transit("stream_shared", "asked_learn", action="ask_learn_gen",
                    args={"du_hash": "<playlist>", "yhat_hash": "<playlist>", "dhat_hash": "<playlist>",
                          "ask_steps": 1000})
teacher.add_transit("asked_learn", "done_learn", action="done_learn_gen")

#### Conditionally determine the next State

Go to the next Stream in the Playlist and check two options:
- if we are not yet in the last round of repetitions, go back to the `stream_shared` state and ask for learning;
- otherwise jump to the `ready_to_ask` state and ask for unsupervised generation.

In [ ]:
teacher.behav.add_state_action("done_learn", action="next_pref_stream")
teacher.behav.add_transit("done_learn", "stream_shared", action="check_pref_stream", args={"what": "not_last_round"})
teacher.behav.add_transit("done_learn", "ready_to_ask", action="check_pref_stream", args={"what": "last_round"})

Coming out from this loop, we ask the Student to generate the signals without supervision.

In [ ]:
teacher.behav.add_transit("ready_to_ask", "asked_gen", action="ask_gen",
                          args={"du_hash": "<playlist>",  "dhat_hash": "<playlist>", "ask_steps": 1000})
teacher.behav.add_transit("asked_gen", "done_gen", action="done_gen")

We need to add another check to loop inside the last repetition of the Playlist to play both songs and to go to the `finished` state after the last one.

In [ ]:
teacher.behav.add_state_action("done_gen", action="next_pref_stream")
teacher.behav.add_transit("done_gen", "ready_to_ask", action="check_pref_stream", args={"what": "not_first"})
teacher.behav.add_transit("done_gen", "finished", action="check_pref_stream", args={"what": "first"})

env.add_agent(teacher)

### Create the new Student

As we have done [here](#Define-a-student-agent), we create a student with the same FSM as before and we add it to the environment.

In [ ]:
student = BasicAgent("Student", model=BasicModel(attributes=env.shared_attributes, lr=0.001, device=device, seed=4), authority=0.0)
student.print_enabled = False  # turn off logging just to speed up the notebook execution
student.add_transit("init", "got_streams", action="get_streams")
student.add_transit("got_streams", "got_agents", action="get_agents")
student.add_transit("got_agents", "teacher_engaged", action="get_engagement", args={"min_auth": 1.0, "max_auth": 1.0})
student.add_transit("teacher_engaged", "got_teacher_streams", action="get_streams")
student.add_transit("got_teacher_streams", "learning", action="do_learn_gen")
student.add_transit("got_teacher_streams", "generated", action="do_gen")
student.add_transit("learning", "got_teacher_streams", action="nop")
student.add_transit("generated", "got_teacher_streams", action="nop")

We add the agent to the environment and run it again.

In [ ]:
env.add_agent(student)
env.run()

## Complete Learning Cycle

This scenario encapsulates the simplest complete learning cycle within NARNIAN:
- the Teacher provides a lesson through recorded signals;
- the Student is tasked with both replicating the provided signal and describing its own output;
- continuous evaluation is performed until the Student’s performance meets a defined threshold;
- the process is sequential, ensuring that the Teacher moves to the next lesson (signal) only after the successful completion of previous one.

### Adapt the Environment

This time we will replace the 3Sin signal with a pure Sine wave, so we need to:
1. remove the Agents before we create the new ones;
2. remove the old signal and add the new one;
3. reset the state of the Environment to the Init state of its FSM.

In [ ]:
env.remove_all_agents()
env.remove_stream(creator=env.name, name='3sin')
env.add_stream(Stream.create(name="sin", creator=env.name, stream=Sin(freq=0.06, phase=0.5, delta=0.1)))
env.behav.reset_state()

### Define the teacher once again

In [ ]:
teacher = BasicAgent("Teacher", model=EmptyModel(), authority=1.0)
teacher.print_enabled = False  # turn off logging just to speed up the notebook execution
teacher.add_transit("init", "got_streams", action="get_streams")
teacher.add_transit("got_streams", "got_agents", action="get_agents")
teacher.add_transit("got_agents", "recording1", action="record",
               args={"stream_hash": env.name + ":sin", "steps": 1000})
teacher.add_transit("recording1", "recording2", action="record",
               args={"stream_hash": env.name + ":square", "steps": 1000})
teacher.add_transit("recording2", "playlist_ready", action="set_pref_streams",
               args={"stream_hashes": [teacher.name + ":recorded1", teacher.name + ":recorded2"], "repeat": 1})
teacher.add_state_action("playlist_ready", action="find_agent_to_engage", args={"min_auth": 0.0, "max_auth": 0.0})
teacher.add_transit("playlist_ready", "student_found", action="send_engagement")
teacher.add_transit("student_found", "playlist_ready", action="nop")
teacher.add_transit("student_found", "student_engaged", action="got_engagement")
teacher.add_transit("student_engaged", "stream_shared", action="share_streams")

#### Learning and Generation phase

The Teacher instructs the Student to learn (both generation and prediction) using the playlist data, and then to generate signals, each for 500 steps.

In [ ]:
teacher.add_transit("stream_shared", "asked_learn", action="ask_learn_gen_and_pred",
               args={"du_hash": "<playlist>", "yhat_hash": "<playlist>", "dhat_hash": "<playlist>",
                     "ask_steps": 1000})
teacher.add_transit("asked_learn", "done_learn", action="done_learn_gen_and_pred")
teacher.add_transit("done_learn", "asked_gen", action="ask_gen_and_pred",
               args={"du_hash": "<playlist>", "ask_steps": 1000})
teacher.add_transit("asked_gen", "done_gen", action="done_gen_and_pred")

#### Evaluation phase

The Teacher evaluates the generated signal by computing the Mean Squared Error (MSE) over 1000 steps. 
Based on the evaluation result:
- If MSE > 0.05, the process loops back to the shared stream state.
- If MSE <= 0.05, it moves to a "good" state.

In [ ]:
teacher.add_state_action("done_gen", action="eval",
                    args={"stream_hash": "<playlist>", "what": "y", "how": "mse", "steps": 1000})
teacher.add_transit("done_gen", "stream_shared", action="compare_eval", args={"cmp": ">", "thres": 0.05})
teacher.add_transit("done_gen", "good", action="compare_eval", args={"cmp": "<=", "thres": 0.05})

#### Playlist progression

When in the "good" state, the Teacher advances to the next preferred stream.
Depending on whether it is the first in the playlist or not, the process either repeats or finishes.

In [ ]:
teacher.add_state_action("good", action="next_pref_stream")
teacher.add_transit("good", "stream_shared", action="check_pref_stream", args={"what": "not_first"})
teacher.add_transit("good", "finished", action="check_pref_stream", args={"what": "first"})

Add the teacher to the environment.

In [ ]:
env.add_agent(teacher)

### Creating the Student

In [ ]:
student = BasicAgent("Student", model=BasicModel(attributes=env.shared_attributes, lr=0.001, device=device, seed=4),
                authority=0.0)
student.print_enabled = False  # turn off logging just to speed up the notebook execution
student.add_transit("init", "got_streams", action="get_streams")
student.add_transit("got_streams", "got_agents", action="get_agents")
student.add_transit("got_agents", "teacher_engaged", action="get_engagement", args={"min_auth": 1.0, "max_auth": 1.0})
student.add_transit("teacher_engaged", "got_teacher_streams", action="get_streams")
student.add_transit("got_teacher_streams", "learning", action="do_learn_gen_and_pred")
student.add_transit("got_teacher_streams", "generated", action="do_gen_and_pred")
student.add_transit("learning", "got_teacher_streams", action="nop")
student.add_transit("generated", "got_teacher_streams", action="nop")

In [ ]:
env.add_agent(student)
env.run()